In [ ]:
!pip install datasets rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.2 MB/s eta 0:00:00


In [1]:
import nltk
from nltk.tokenize import sent_tokenize
# Ensure that the necessary NLTK resources are downloaded
nltk.download('punkt')

def lead_5_summarization(text):
    """
    Summarize the text by extracting the first three sentences.
    Args:
    text (str): The text to be summarized.

    Returns:
    str: The summary of the text.
    """
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Extract the first three sentences or fewer if the text is shorter
    summary = ' '.join(sentences[:5])

    return summary

# Example text
example_text = """
COVID-19 is an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.
"""

# Generate summary
summary = lead_5_summarization(example_text)
print("Summary:")
print(summary)

Summary:

COVID-19 is an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peeyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("allenai/cord19", "fulltext", "abstract", trust_remote_code=True)

In [3]:
import numpy as np
def select_datapints(dataset):
    dataset['select'] = (len(dataset['fulltext']) > 0) and (len(dataset['abstract']) > 0)
    return dataset

raw_datasets['train'] = raw_datasets['train'].map(select_datapints)
indices_required = np.where(np.array(raw_datasets['train']['select']) == True)[0]
raw_datasets['train'] = raw_datasets['train'].select(indices_required)
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['cord_uid', 'sha', 'source_x', 'title', 'doi', 'abstract', 'publish_time', 'authors', 'journal', 'url', 'fulltext', 'select'],
        num_rows: 105097
    })
})

In [4]:
raw_datasets = raw_datasets.remove_columns(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'publish_time', 'authors', 'journal', 'url'])

import gc
gc.collect()

from datasets import DatasetDict
train_testvalid = raw_datasets['train'].train_test_split(test_size=0.2)

# Further split the test+validation into test and validation sets evenly
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

gc.collect()

test_valid = test_valid['test']

In [8]:

def get_rouge_scores(document):
    import rouge

    from nltk.tokenize import sent_tokenize
    # Ensure that the necessary NLTK resources are downloaded
    
    def lead_5_summarization(text):
        """
        Summarize the text by extracting the first three sentences.
        Args:
        text (str): The text to be summarized.

        Returns:
        str: The summary of the text.
        """
        # Tokenize the text into sentences
        sentences = sent_tokenize(text)

        # Extract the first three sentences or fewer if the text is shorter
        summary = ' '.join(sentences[:5])

        return summary
    
    generated_text = lead_5_summarization(document['fulltext'])
    sentence_rouge_score = rouge.Rouge().get_scores(generated_text,
                                                    document['abstract'])[0]
    for rog in sentence_rouge_score:
        document[rog] = sentence_rouge_score[rog]['f']


    return document



In [9]:
test = test_valid.map(get_rouge_scores, num_proc=8)

np.mean(test['rouge-1']), np.mean(test['rouge-2']), np.mean(test['rouge-l'])

Map (num_proc=8):   0%|          | 0/10510 [00:00<?, ? examples/s]

(0.2666797759746475, 0.11413433837889586, 0.24606581852237497)